In [1]:
import sys
import gc
import wandb

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [2]:
import sklearn
import lightgbm

print(sklearn.__version__)
print(lightgbm.__version__)

## log into wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


1.2.2
3.3.2


wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin


True

In [3]:
test = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/test_essays.csv')
sub = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/sample_submission.csv')
org_train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/train_essays.csv')

In [4]:
train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/daigt/train_v2_drcat_02.csv', sep=',')
display(train.shape);

(44868, 5)

#### Read and append the generated data

In [5]:
train_gen_data = pd.read_parquet('/home/peng_sun2/s3shared/kaggle/llm-2023/external_data/gen_data_21122023.parquet')
display(train_gen_data.shape)

(18894, 5)

In [6]:
train = train.drop_duplicates(subset = ['text'])
train.reset_index(drop = True, inplace = True)

### append the generated data
# train = pd.concat([train, train_gen_data], axis = 0).reset_index(drop = True)
# display(train.shape)

In [7]:
train_0 = train.loc[train['label'] ==0, :].reset_index(drop = True)
train_1 = train.loc[train['label'] ==1, :].reset_index(drop = True)

In [8]:
train_0.shape

(27371, 5)

In [9]:
train_1.shape

(17497, 5)

In [10]:
# min_length = min(len(train_0), len(train_1))
# min_length

In [11]:
# sorted_train= pd.DataFrame()
# for i in tqdm(range(min_length), total = min_length):
#     sorted_train = pd.concat([sorted_train, pd.DataFrame(train_0.iloc[i,:]).T], axis = 0)
#     sorted_train = pd.concat([sorted_train, pd.DataFrame(train_1.iloc[i,:]).T], axis = 0)

In [12]:
# if len(train_0) > min_length:
#     sorted_train = pd.concat([sorted_train, train_0.iloc[min_length:, :]], axis = 0)
# elif len(train_1) > min_length:
#     sorted_train = pd.concat([sorted_train, train_1.iloc[min_length:, :]], axis = 0)

In [13]:
from tqdm import tqdm 
def sort_train(train):
    train_0 = train.loc[train['label'] ==0, :].reset_index(drop = True)
    train_1 = train.loc[train['label'] ==1, :].reset_index(drop = True)
    
    min_length = min(len(train_0), len(train_1))
    
    sorted_train= pd.DataFrame()
    for i in tqdm(range(min_length), total = min_length):
        sorted_train = pd.concat([sorted_train, pd.DataFrame(train_0.iloc[i,:]).T], axis = 0)
        sorted_train = pd.concat([sorted_train, pd.DataFrame(train_1.iloc[i,:]).T], axis = 0)
    
    if len(train_0) > min_length:
        sorted_train = pd.concat([sorted_train, train_0.iloc[min_length:, :]], axis = 0)
    elif len(train_1) > min_length:
        sorted_train = pd.concat([sorted_train, train_1.iloc[min_length:, :]], axis = 0)
        
    del train_0, train_1, train
    sorted_train['label'] = sorted_train['label'].astype(np.int32)
    return sorted_train.reset_index(drop = True)

In [14]:
display(train.shape)
train.head(10)

(44868, 5)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
5,Cell phone use should not be legal while drivi...,0,Phones and driving,persuade_corpus,False
6,Phones and Driving\n\nDriving is a good way to...,0,Phones and driving,persuade_corpus,False
7,PHONES AND DRIVING\n\nIn this world in which w...,0,Phones and driving,persuade_corpus,False
8,People are debating whether if drivers should ...,0,Phones and driving,persuade_corpus,False
9,Texting and driving\n\nOver half of drivers in...,0,Phones and driving,persuade_corpus,False


In [15]:
train = sort_train(train)

100%|██████████| 17497/17497 [00:46<00:00, 376.03it/s]


In [16]:
display(train.shape)
train.head(10)

(44868, 5)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,"In recent years, technology has had a profoun...",1,Car-free cities,mistral7binstruct_v2,True
2,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
3,I strongly believe that meditation and mindful...,1,Distance learning,llama_70b_v1,False
4,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
5,One way school administrators can attempt to c...,1,Cell phones at school,chat_gpt_moth,False
6,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
7,While summer is meant as a break from the regu...,1,Summer projects,darragh_claude_v7,False
8,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
9,The use of Facial Action Coding System (FACS) ...,1,Facial action coding system,darragh_claude_v6,True


In [17]:
LOWERCASE = False
VOCAB_SIZE = 30522

In [18]:
## Create byte-pair encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token='[UNK]'))

In [19]:
## adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()]+ [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

In [20]:
# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size = VOCAB_SIZE, special_tokens = special_tokens)

In [21]:
# creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i:i+1000]['text']
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer = trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyarrow/pandas_compat.py:358: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


100%|██████████| 3/3 [00:00<00:00, 1422.28it/s]


100%|██████████| 44868/44868 [02:03<00:00, 363.59it/s]


In [22]:
tokenized_texts_test[1]

['ĠBbb', 'Ġccc', 'Ġddd', '.']

In [23]:
def dummy(text):
    return text

In [24]:
vectorizer = TfidfVectorizer(ngram_range = (3, 5), lowercase=False, sublinear_tf = True, analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None,
                            strip_accents = 'unicode',
                            dtype = np.float32)
vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, 
                             #vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode',
                            dtype = np.float32
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)
## get the vocabulary of the training dataset
train_vocab = vectorizer.vocabulary_

del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


0

### Convert the type from numpy.float64 to numpy.float16

In [25]:
# import numpy as np
# from scipy.sparse import csr_matrix

# ## convert data for tf_train
# data  = tf_train.data.astype(np.float16)
# indices = tf_train.indices
# indptr = tf_train.indptr
# tf_train = csr_matrix((data, indices, indptr), shape = tf_train.shape)


# ## convert data for tf_test
# data = tf_test.data.astype(np.float16)
# indices = tf_test.indices
# indptr = tf_test.indptr
# tf_test = csr_matrix((data, indices, indptr), shape = tf_test.shape)

# del data, indices, indptr
# gc.collect()

## Start training

In [26]:
# ### save train_vocab
# import joblib
# model_path = '/home/peng_sun2/s3shared/kaggle/llm-2023/baseline1/'
# joblib.dump(train_vocab, f'{model_path}vocab_train.pkl')

In [27]:
# # Check if it's in scoring stage
# if len(test.text.values) <= 5:
#     # if not, just sample submission
#     sub.to_csv('submission.csv', index=False)
# else:
# otherwise, run fitting process

# p6={
#     'n_iter': 1500,
#     'verbose': -1,
#     'objective': 'binary',
#     'metric': 'auc',
#     'learning_rate': 0.05073909898961407, 
#     'colsample_bytree': 0.726023996436955, 
#     'colsample_bynode': 0.5803681307354022, 
#     'lambda_l1': 8.562963348932286, 
#     'lambda_l2': 4.893256185259296, 
#     'min_data_in_leaf': 115, 
#     'max_depth': 123, 
#     'max_bin': 250,
#     'device': 'gpu'
# }

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'validatoin_auc', 'goal': 'maximize'},
    'parameters': {
        'alpha': {'min': 0.0001, 'max': 0.01},
        'loss': {'values':['hinge', 'log', 'modified_huber']},
        'eta0': {'min': 0.01, 'max': 0.1},
        'max_iter': {'min': 1000, 'max': 10000}
    }
}

sweep_id = wandb.sweep(sweep_config, project="llm-2023-SGD-hyperparameter")


Create sweep with ID: wyi8yb2f
Sweep URL: https://wandb.ai/peng_sun/llm-2023-SGD-hyperparameter/sweeps/wyi8yb2f


In [28]:
def split_range_into_parts(start, end, num_parts):
    splits = np.linspace(start, end, num_parts + 1)
    ranges = [(int(np.floor(splits[i])), int(np.ceil(splits[i + 1]))) for i in range(num_parts)]
    return np.array(ranges)

In [29]:
def increment_train_catboost(tf_train, y_train, cat, num_parts):
    split_ranges = split_range_into_parts(0, tf_train.shape[0], num_parts);
    #print(split_ranges)
    
    ## incrementally train catboost
    for i, (start, end) in enumerate(split_ranges):
        print('start, end ', start, ' ', end)
        train_X_split = tf_train[start:end, :]
        train_y_split = y_train[start:end]
        
        #print(train_X_split.shape, train_y_split.shape)
        if i ==0:
            cat.fit(train_X_split, train_y_split)
        else: 
            cat.fit(train_X_split, train_y_split, init_model = cat)
        print(f'part {i} is finished ...')
    return cat
    

In [30]:
y_train = train['label'].values

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
## cv split
skf = StratifiedGroupKFold(n_splits = 5)
skf.get_n_splits(tf_train, y_train, groups =train['prompt_name'])

def train_sweep():
    with wandb.init() as run:
        config = wandb.config
        for i, (train_idx, test_idx) in enumerate(skf.split(tf_train, y_train, groups = train['prompt_name'])):
            train_X = tf_train[train_idx]
            eval_X = tf_train[test_idx];
            train_y = y_train[train_idx]
            eval_y = y_train[test_idx];
 
            model = SGDClassifier(
                alpha = config.alpha,
                loss = config.loss,
                eta0 = config.eta0,
                max_iter = config.max_iter
            )
            model.fit(train_X, train_y);
            eval_preds = model.predict_proba(eval_X)[:, 1]
            fpr, tpr, thresholds = metrics.roc_curve(eval_y, eval_preds, pos_label = 1)
            eval_auc = metrics.auc(fpr, tpr)
            wandb.log({
                'validatoin_auc': eval_auc
            });
wandb.agent(sweep_id, function = train_sweep)

wandb: Agent Starting Run: o7lttkq3 with config:
wandb: 	alpha: 0.004908504150149459
wandb: 	eta0: 0.027093758025708715
wandb: 	loss: modified_huber
wandb: 	max_iter: 4210
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


validatoin_auc,██▇▁▇
validatoin_auc,0.97974


wandb: Agent Starting Run: yihjp15o with config:
wandb: 	alpha: 0.008979464566136266
wandb: 	eta0: 0.03823282561522628
wandb: 	loss: log
wandb: 	max_iter: 7731
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


/home/peng_sun2/.local/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/peng_sun2/.local/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/peng_sun2/.local/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/peng_sun2/.local/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


validatoin_auc,▇█▆▁▄
validatoin_auc,0.94332


...

wandb: Agent Starting Run: godcn1xu with config:
wandb: 	alpha: 0.00028250942214293906
wandb: 	eta0: 0.018961370238035104
wandb: 	loss: modified_huber
wandb: 	max_iter: 1370
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


validatoin_auc,███▁█
validatoin_auc,0.99255


wandb: Agent Starting Run: o4s7fs89 with config:
wandb: 	alpha: 0.00017211102227684567
wandb: 	eta0: 0.06433252509244095
wandb: 	loss: modified_huber
wandb: 	max_iter: 2221
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


validatoin_auc,███▁█
validatoin_auc,0.99339


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: yyelhxe9 with config:
wandb: 	alpha: 0.00028454198645635846
wandb: 	eta0: 0.02291995001785229
wandb: 	loss: modified_huber
wandb: 	max_iter: 7888
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [34]:
wandb.finish()

### Best parameters

* colsample_bynode = 0.9669
* colsample_bytree = 0.9645
* lambda_l1 = 0.7217
* lambda_l2 = 3.207
* learning_rate = 0.09574
* max_bin = 184
* max_depth = 62
* min_data_in_leaf = 111
